### **This lab aims to collect data for the feature-based volume classification model. Features including the width of the bounding box, the height of the bounding box, the type of the recyclable object, whether smashed or not, etc...**

In [ ]:
import os
import cv2
import pandas as pd
import keras
import numpy as np
import tensorflow as tf
from keras.utils import normalize
import matplotlib.pyplot as plt

In [ ]:
!pip install gdown

In [ ]:
# Importing the dataset that will be used to collect the data for the volume classification model
!gdown https://drive.google.com/uc?id=1FpJBz9gSO0xLCwBoZ5rQUvq-jSoGjBrx

In [ ]:
# Unzipping the dataset compressed file
!unzip dataset_for_volume_classification_model.zip

In [ ]:
# Remove the  compressed dataset file(no need for the compressed files)
!rm dataset_for_volume_classification_model.zip

In [ ]:
dataset_directory = "dataset_for_volume_classification_model"

In [ ]:
#These are all the classes(categories) we want our volume classification model to classify later on
os.listdir(dataset_directory)

## **Extracting the width of the bounding box, the height of the bounding box, the surface of the bounding box and the type of the recyclable object(bottle, can)**

In [ ]:
# Importing the object detection model (tflite version)
!gdown https://drive.google.com/uc?id=1--pJA_MACxCL04uw7u_zqmJCSIi7faYp

In [ ]:
# Unzipping the object detection model compressed file
!unzip object_detection_model_tflite.zip

In [ ]:
# Remove the compressed object detection model file(no need for the compressed files)
!rm object_detection_model_tflite.zip

In [ ]:
!touch object_detection_model_tflite/labels.txt

In [ ]:
with open('object_detection_model_tflite/labels.txt', 'w') as f:
    f.write("bottle\n")
    f.write("cup\n")
    f.write("nocup\n")
    f.write("can")

In [ ]:
import re

In [ ]:
!pip install tflite-runtime

In [ ]:
from tflite_runtime.interpreter import Interpreter

In [ ]:
def load_labels(path='object_detection_model_tflite/labels.txt'):
  """Loads the labels file. Supports files with or without index numbers."""
  with open(path, 'r', encoding='utf-8') as f:
    lines = f.readlines()
    labels = {}
    for row_number, content in enumerate(lines):
      pair = re.split(r'[:\s]+', content.strip(), maxsplit=1)
      if len(pair) == 2 and pair[0].strip().isdigit():
        labels[int(pair[0])] = pair[1].strip()
      else:
        labels[row_number] = pair[0].strip()
  return labels

In [ ]:
def set_input_tensor(interpreter, image):
  """Sets the input tensor."""
  tensor_index = interpreter.get_input_details()[0]['index']
  input_tensor = interpreter.tensor(tensor_index)()[0]
  input_tensor[:, :] = np.expand_dims(image, axis=0)

In [ ]:
def get_output_tensor(interpreter, index):
  """Returns the output tensor at the given index."""
  output_details = interpreter.get_output_details()[int(index)]
  tensor = np.squeeze(interpreter.get_tensor(output_details['index']))
  return tensor

In [ ]:
def detect_objects(interpreter, image, threshold):
  """Returns a list of detection results, each a dictionary of object info."""
  set_input_tensor(interpreter, image)
  interpreter.invoke()
  # Get all output details
  boxes = get_output_tensor(interpreter, 1)
  classes = get_output_tensor(interpreter, 3)
  scores = get_output_tensor(interpreter, 0)
  count = int(get_output_tensor(interpreter, 2))
  results = []
  for i in range(count):
    if scores[i] >= threshold:
      result = {
          'bounding_box': boxes[i],
          'class_id': classes[i],
          'score': scores[i]
      }
      results.append(result)
  return results

In [ ]:
labels = load_labels()
interpreter = Interpreter('object_detection_model_tflite/saved_model/object_detection_model.tflite')
interpreter.allocate_tensors()
_, input_height, input_width, _ = interpreter.get_input_details()[0]['shape']

In [ ]:
def object_detection(img_path):
    img = cv2.imread(img_path)
    img = np.array(img)
    img = cv2.resize(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), (512,512))
    results = detect_objects(interpreter, img, 0.4)
    return results

In [ ]:
labels = ['bottle', 'cup', 'nocup', 'can']

In [ ]:
data = pd.DataFrame(columns=["ymin", "xmin", "ymax", "xmax"])

In [ ]:
detected_objects = []
detected_boxes = []
for directory in os.listdir(dataset_directory):
    for i, img in enumerate(os.listdir(os.path.join(dataset_directory, directory))):
        img_path = os.path.join(dataset_directory, directory, img)
        results = object_detection(img_path)
        for result in results:
            detected_objects.append(labels[int(result['class_id'])])
            detected_boxes.append(result['bounding_box'])
            
for detected_box in detected_boxes:
    df = pd.DataFrame([detected_box], columns=["ymin", "xmin", "ymax", "xmax"])
    data = pd.concat([data, df], axis=0, ignore_index=True)
   
data = data.assign(detected_object=pd.Series(detected_objects).values)

In [ ]:
data.head()

In [ ]:
data = data[((data["detected_object"] == "bottle") | (data["detected_object"] == "can"))]

In [ ]:
df = pd.DataFrame()
width = data['xmax'] - data['xmin']
df = df.assign(width=pd.Series(width).values)
height = data['ymax'] - data['ymin']
df = df.assign(height=pd.Series(height).values)
surface = width*height
df = df.assign(surface=pd.Series(surface).values)
detected_object = data['detected_object']
df = df.assign(detected_object=pd.Series(detected_object).values)

In [ ]:
df.head()

## **Extracting whether or not the recyclable object is defective (smashed)**

In [ ]:
# Importing the defective_non_defective classification model(tflite version)
!gdown https://drive.google.com/uc?id=1-0WB4vJoYlaD8ise2n9LO65JCSsTHQxX

In [ ]:
# Unzipping the defective_non_defective classification model compressed file
!unzip defective_non_defective_classification_model_tflite.zip

In [ ]:
# Removing the defective_non_defective classification model compressed file(no need for the compressed files)
!rm defective_non_defective_classification_model_tflite.zip

In [ ]:
interpreter = tf.lite.Interpreter(model_path="defective_non_defective_classification_model_tflite.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data.
input_shape = input_details[0]['shape']

In [109]:
classes = ['non_defective', 'defective']

In [110]:
def defective_non_defective(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), (512,512))
    img = np.array(img)
    img = img/255.0
    img = img.astype(np.float32)
    img = np.expand_dims(img, axis=0)
    interpreter.set_tensor(input_details[0]['index'], img)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    predicted_class = np.argmax(output_data)
    return classes[predicted_class]

In [ ]:
results = []
for directory in os.listdir(dataset_directory):
    for i, img in enumerate(os.listdir(os.path.join(dataset_directory, directory))):
        img_path = os.path.join(dataset_directory, directory, img)
        predict = defective_non_defective(img_path)
        results.append(predict)

In [ ]:
df = df.assign(object_condition=pd.Series(results).values)

In [ ]:
df.head()

In [ ]:
volume = []
for directory in os.listdir(dataset_directory):
    for i, img in enumerate(os.listdir(os.path.join(dataset_directory, directory))):
        volume.append(directory)
volume = [float(v) for v in volume]

In [101]:
df_v1 = df.copy()

In [103]:
df_v1 = df_v1.assign(volume=pd.Series(volume).values)

In [104]:
df_v1.head()

,width,height,surface,detected_object,object_condition,volume
0,0.405624,0.271085,0.109959,bottle,defective,0.5
1,0.395052,0.267947,0.105853,bottle,defective,0.5
2,0.440588,0.209557,0.092328,bottle,non_defective,0.5
3,0.450576,0.264297,0.119086,bottle,defective,0.5
4,0.419848,0.212147,0.089069,bottle,non_defective,0.5


In [106]:
df_v1.to_csv('dataset_for_volume_classification_model_v1.csv', encoding='utf-8', index=False)

 ## **Extracting the brand of the recyclable object**

In [114]:
# Importing the brand classification model(tflite version)
!gdown https://drive.google.com/uc?id=1bv48M1nLuxlCMZp5Qm1ZtqYAnLTU0K1t

Downloading...
From (uriginal): https://drive.google.com/uc?id=1bv48M1nLuxlCMZp5Qm1ZtqYAnLTU0K1t
From (redirected): https://drive.google.com/uc?id=1bv48M1nLuxlCMZp5Qm1ZtqYAnLTU0K1t&confirm=t&uuid=62f082f5-b689-49a8-a1ab-9644646ff736
To: /kaggle/working/brand_classification_model_tflite.zip
100%|████████████████████████████████████████| 688M/688M [00:12<00:00, 55.3MB/s]


In [115]:
# Unzipping the brand classification model compressed file
!unzip brand_classification_model_tflite.zip

Archive:  brand_classification_model_tflite.zip
  inflating: volume_brand_classification_model_tflite.tflite  


In [116]:
# Remove the brand classification model compressed file(no need for the compressed files)
!rm brand_classification_model_tflite.zip

In [117]:
interpreter = tf.lite.Interpreter(model_path="volume_brand_classification_model_tflite.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data.
input_shape = input_details[0]['shape']

In [118]:
classes = [
    'BOTTLE_0,5_SAFIA',
     'BOTTLE_0,5_ROYALE',
     'BOTTLE_1,0_MIRA',
     'BOTTLE_1,5_MELLITI',
     'CAN_0,24_SPRITE',
     'BOTTLE_0,5_CRISTALINE',
     'BOTTLE_1,5_DELICE',
     'BOTTLE_1,5_BOGACIDRE',
     'BOTTLE_0,5_MARWA',
     'CAN_0,24_BOGACITRON',
     'BOTTLE_2,0_DIMA',
     'BOTTLE_1,5_SAFIA',
     'BOTTLE_1,5_MARWA',
     'BOTTLE_0,5_TIBA',
     'CAN_0,24_COCA',
     'BOTTLE_2,0_DENYA',
     'CAN_0,24_FANTA',
     'BOTTLE_1,5_TIJEN',
     'CAN_0,24_ORANGINA',
     'BOTTLE_0,5_TIJEN',
     'BOTTLE_0,5_COCA',
     'BOTTLE_0,5_BEYA',
     'BOTTLE_2,0_FOURAT',
     'BOTTLE_0,5_DELICE',
     'BOTTLE_0,5_AQUALINE',
     'CAN_0,24_APLA',
     'BOTTLE_1,5_SABRINE',
     'BOTTLE_1,5_BARGOU',
     'BOTTLE_0,5_MIRA',
     'BOTTLE_1,5_COCA',
     'BOTTLE_0,5_DIMA',
     'CAN_0,24_BOGACIDRE']

In [119]:
def brand_classification(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), (512,512))
    img = np.array(img)
    img = img/255.0
    img = img.astype(np.float32)
    img = np.expand_dims(img, axis=0)
    interpreter.set_tensor(input_details[0]['index'], img)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    predicted_class = np.argmax(output_data)
    return classes[predicted_class]

In [122]:
brands = []
for directory in os.listdir(dataset_directory):
    for i, img in enumerate(os.listdir(os.path.join(dataset_directory, directory))):
        img_path = os.path.join(dataset_directory, directory, img)
        brand = brand_classification(img_path)
        brands.append(brand)
brands = [brand.split('_')[2] for brand in brands]

In [129]:
df_v2 = df.copy()

In [130]:
df_v2 = df_v2.assign(brand=pd.Series(brands).values)

In [131]:
df_v2 = df_v2.assign(volume=pd.Series(volume).values)

In [132]:
df_v2.head()

,width,height,surface,detected_object,object_condition,brand,volume
0,0.405624,0.271085,0.109959,bottle,defective,DIMA,0.5
1,0.395052,0.267947,0.105853,bottle,defective,TIJEN,0.5
2,0.440588,0.209557,0.092328,bottle,non_defective,COCA,0.5
3,0.450576,0.264297,0.119086,bottle,defective,MIRA,0.5
4,0.419848,0.212147,0.089069,bottle,non_defective,TIJEN,0.5


In [133]:
df_v2.to_csv('dataset_for_volume_classification_model_v2.csv', encoding='utf-8', index=False)